In [2]:
import scipy.sparse
import itertools
import pandas as pd
from utils import *
from collections import defaultdict, OrderedDict
import numpy as np
from sklearn.metrics import confusion_matrix

In [3]:
velo_fields = {
    'cca': ['common carotid', 'cca'],
    'bulb': ['carotid bulb'],
    'eca': ['external carotid'],
    'ica': ['internal carotid', 'ica']
}
prefix = {
    'p': ['proximal', 'prox'],
    'd': ['distal', 'dist'],
    'm': ['middle', 'mid'],
}
fuzzyName = {
    'study': ['study', 'examination'],
    'findings': ['findings'],
    'impression': ['impression'],
    'history': ['history', 'indication'],
    'comparison': ['comparison'],
    'technique': ['technique'],
    'signed by': ['signed by'],
}
orderField = [u'study',
              u'history',
              u'comparison',
              u'technique',
              u'findings',
              u'impression', 
              u'signed by',
             ]

--------

In [4]:
filename = 'Data/upto1526.xlsx'
df_raw = pd.read_excel(open(filename, 'r'))

In [8]:
# get data frame
ps = Parser(fuzzyName)
ps.parser(df_raw)
df = ps.df
for idx, row in df['findings'].iteritems():
    try:
        text, velos = parse_findings(row)
        df.set_value(idx, 'findings', text)
        for n, v in velos:
            df.set_value(idx, n, v)
    except:
        pass
discardField = ['Report Text']
foo = [item for item in df.columns.tolist() if item not in orderField+discardField]
foo.sort()
CORE_COL = orderField + foo
df = df[CORE_COL]
df = pd.concat([df_raw[['Past', 'Present']], df[CORE_COL]], axis=1)

In [5]:
# df = null2empty(df, 'history' 'impression')

In [6]:
# write to documents
#df[col].to_excel('reports(07192017).xlsx')

---------

# Classification Model

## Use findings and impression to predict present and pass

In [9]:
df = null2empty(df, ['history', 'impression', 'comparison'])
print df.shape

(1525, 47)


/Users/vincent/.virtualenvs/env1/lib/python2.7/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [28]:
#for PAST
fields = ['history', 'impression', 'findings', 'comparison']
foo = df[~df['Past'].isnull() & df['Past'] != 0].sample(frac=1, random_state=1)
df_train = foo.iloc[:900]
y_train = np.array(df_train['Past'].astype(int))
df_test = foo.iloc[900:]
y_test = np.array(df_test['Past'].astype(int))
obj = Df2TFIDF()
obj.fit(df_train, fields, ngram=5, min_count=5)
output_train = obj.transform(df_train, fields)
output_test = obj.transform(df_test, fields)

In [29]:
# concatenate sparse matrices of all fields
from scipy.sparse import hstack
x_train = hstack([foo['bow_tfidf'] for foo in output_train.itervalues()])
x_test = hstack([foo['bow_tfidf'] for foo in output_test.itervalues()])
print x_train.shape
print x_test.shape

(900, 9253)
(625, 9253)


In [25]:
# #for PRESENT
# fields = ['history','impression','findings', 'comparison']
# msk1 = np.random.rand(len(df[~df['Present'].isnull() & df['Present'] != 0])) < 0.7
# df_train = df[~df['Present'].isnull() & df['Present'] != 0][msk1]
# y_train = np.array(df_train['Present'].astype(int))
# text_train = df2text(df_train, fields=fields)
# df_test = df[~df['Present'].isnull() & df['Present'] != 0][~msk1]
# y_test = np.array(df_test['Present'].astype(int))

In [30]:
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
# svm
#clf = LinearSVC()
#clf = SVC()
clf = LogisticRegression()
clf.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [31]:
print clf.score(x_train, y_train)
print clf.score(x_test, y_test)

0.956666666667
0.9184


In [18]:
y_train_pred = clf.predict(data_train)
y_test_pred = clf.predict(data_test)
CM_train = confusion_matrix(y_train, y_train_pred)
CM_test = confusion_matrix(y_test, y_test_pred)
print CM_train[0][0] #TN
print CM_train[1][0] #FN
print CM_train[1][1] #TP
print CM_train[0][1] #FP

print CM_test[0][0] #TN
print CM_test[1][0] #FN
print CM_test[1][1] #TP
print CM_test[0][1] #FP

734
95
191
4
240
8
109
55


In [202]:
y_hat = np.asarray((data_train[:, 0] > 0).todense()) * 2 - 1
sum(y_hat.squeeze() - np.array(y_train) != 0)

931

In [203]:
y_hat = np.asarray((data_test[:,0] > 0).todense()) * 2 - 1
sum(y_hat.squeeze() - np.array(y_test) != 0)

226

In [204]:
len(y_test)

297

In [205]:
type(y_train[0])

numpy.int64